# Лабораторная работа 4

1. Выбрать бизнес-цели для набора данных по Вашему варианту задания для решения задач регрессии и классификации.

2. Определить достижимый уровень качества модели для каждой задачи.

3. Выбрать ориентир для каждой задачи.

4. Выполнить анализ применимости алгоритмов обучения с учителем для решения поставленных задач с учетом свойств алгоритмов.

5. Выбрать не менее трех моделей для каждой задачи.

6. Построить конвейер.

7. Реализовать настройку гиперпараметров для каждой модели.

8. Обучить модели.

9. Оценить качество моделей для решения каждой из задач. Выбор метрик необходимо обосновать.

10. Оценить смещение и дисперсию лучших моделей.

11. Все выводы и программный код должны быть оформлены в виде ноутбука. Для выполнения данной лабораторной работы следует создать новый файл-ноутбук.

12. Разобраться с основными возможностями библиотеки scikit-learn (https://scikit-learn.org/1.5/).


# Ход работы

## 1. Выбрать бизнес-цели для набора данных по Вашему варианту задания для решения задач регрессии и классификации.

1) В качестве бизнес цели рассмотрим задачу определения ежедневных продаж (Целевой признак) от Площади и кол-ва товаров - регрессия
2) Для задачи классификации рассмотрим задачу определения кол-ва людей в магазине (Дискретезируем признак)

## 2. Определить достижимый уровень качества модели для каждой задачи.

Разметка не требует сложных данных от пользователя

## 3. Выбрать ориентир для каждой задачи
1) Алгоритм случайного предсказания
2) Предсказательная способность человека

## 4. Выполнить анализ применимости алгоритмов обучения с учителем для решения поставленных задач с учетом свойств алгоритмов.

1) Данные размечены
2) Ориентир выбран
3) Данные будут разбиты на наборы
4) Статическим похожи

## 5. Выбрать не менее трех моделей для каждой задачи.

## 6. Построить конвейер.


In [206]:
from pathlib import Path
import pandas as pd
from sklearn import set_config

set_config(transform_output="pandas")
random_state = 1

dataset_path = Path('../datasets/Stores.csv')
df = pd.read_csv(dataset_path, index_col="Store ID ")

In [207]:
from extra.utils import split_into_train_val_test

X_train, X_val, X_test, y_train, y_val, y_test  = split_into_train_val_test(
    df, stratify_colname="Store_Sales", frac_train=0.80, frac_val=0, frac_test=0.20, random_state=random_state
)


display("X_train", X_train)
display("y_train", y_train)

display("X_test", X_test)
display("y_test", y_test)

'X_train'

,Store_Area,Items_Available,Daily_Customer_Count,Store_Sales
Store ID,,,,
120,1682,2017,780,49390
266,1240,1492,940,66840
356,1269,1511,500,54360
877,1259,1493,100,88270
782,1549,1871,740,74080
...,...,...,...,...
716,1990,2384,1110,64690
768,1119,1345,790,78060
73,891,1073,630,67370


'y_train'

,Store_Sales
Store ID,
120,49390
266,66840
356,54360
877,88270
782,74080
...,...
716,64690
768,78060
73,67370


'X_test'

,Store_Area,Items_Available,Daily_Customer_Count,Store_Sales
Store ID,,,,
85,1287,1525,1200,90180
689,1160,1401,900,35360
143,1214,1456,530,48950
795,1057,1280,610,41050
745,854,1018,660,77740
...,...,...,...,...
530,1554,1844,670,38530
464,1172,1411,460,74550
579,1701,2034,660,62180


'y_test'

,Store_Sales
Store ID,
85,90180
689,35360
143,48950
795,41050
745,77740
...,...
530,38530
464,74550
579,62180


In [208]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

columns_to_drop = ["Daily_Customer_Count", "Store_Sales"]
num_columns = [
    column
    for column in df.columns
    if column not in columns_to_drop and df[column].dtype != "object"
]
num_imputer = SimpleImputer(strategy="median")
num_scaler = StandardScaler()
preprocessing_num = Pipeline(
    [
        ("imputer", num_imputer),
        ("scaler", num_scaler),
    ]
)

features_preprocessing = ColumnTransformer(
    verbose_feature_names_out=False,
    transformers=[
        ("prepocessing_num", preprocessing_num, num_columns),
    ],
    remainder="passthrough"
)

drop_columns = ColumnTransformer(
    verbose_feature_names_out=False,
    transformers=[
        ("drop_columns", "drop", columns_to_drop),
    ],
    remainder="passthrough",
)

pipeline_end = Pipeline(
    [
        ("features_preprocessing", features_preprocessing),
        ("drop_columns", drop_columns),
    ]
)

In [209]:
preprocessing_result = pipeline_end.fit_transform(X_train)
preprocessed_df = pd.DataFrame(
    preprocessing_result,
    columns=pipeline_end.get_feature_names_out(),
)

preprocessed_df

,Store_Area,Items_Available
Store ID,,
120,0.796288,0.794194
266,-0.965760,-0.951834
356,-0.850150,-0.888644
877,-0.890016,-0.948508
782,0.266079,0.308632
...,...,...
716,2.024140,2.014751
768,-1.448130,-1.440722
73,-2.357060,-2.345330


In [210]:
from sklearn import linear_model, tree, neighbors, ensemble, neural_network
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures

class_models = {
    "linear": {"model": linear_model.LinearRegression(n_jobs=-1)},
    "linear_poly": {
        "model": make_pipeline(
            PolynomialFeatures(degree=2),
            linear_model.LinearRegression(fit_intercept=False, n_jobs=-1),
        )
    },
    "linear_interact": {
        "model": make_pipeline(
            PolynomialFeatures(interaction_only=True),
            linear_model.LinearRegression(fit_intercept=False, n_jobs=-1),
        )
    },
    "ridge": {"model": linear_model.RidgeCV()},
    "decision_tree": {
        "model": tree.DecisionTreeRegressor(max_depth=7, random_state=random_state)
    },
    "knn": {"model": neighbors.KNeighborsRegressor(n_neighbors=7, n_jobs=-1)},
    "random_forest": {
        "model": ensemble.RandomForestRegressor(
            max_depth=7, random_state=random_state, n_jobs=-1
        )
    },
    "mlp": {
        "model": neural_network.MLPRegressor(
            activation="tanh",
            hidden_layer_sizes=(3,),
            max_iter=500,
            early_stopping=True,
            random_state=random_state,
        )
    },
}

In [211]:
import math
from pandas import DataFrame
from sklearn import metrics

def run_regression(
    model: Pipeline,
    X_train: DataFrame,
    X_test: DataFrame,
    y_train: DataFrame,
    y_test: DataFrame,
) -> dict:
    result = {}
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    result["pipeline"] = model
    result["train_preds"] = y_train_pred
    result["preds"] = y_test_pred

    result["RMSE_train"] = math.sqrt(metrics.mean_squared_error(y_train, y_train_pred))
    result["RMSE_test"] = math.sqrt(metrics.mean_squared_error(y_test, y_test_pred))
    result["RMAE_test"] = math.sqrt(metrics.mean_absolute_error(y_test, y_test_pred))
    result["R2_test"] = metrics.r2_score(y_test, y_test_pred)

    return result

In [212]:
for model_name in class_models.keys():
    print(f"Model: {model_name}")
    model = class_models[model_name]["model"]

    pipeline = Pipeline([("pipeline", pipeline_end), ("model", model)])
    
    pipeline.fit(
        X_train, y_train.values.ravel()
    )
    
    class_models[model_name] = run_regression(
        pipeline, X_train, X_test, y_train, y_test
    )

Model: linear
Model: linear_poly
Model: linear_interact
Model: ridge
Model: decision_tree
Model: knn
Model: random_forest
Model: mlp


In [213]:
reg_metrics = pd.DataFrame.from_dict(class_models, "index")[
    ["RMSE_train", "RMSE_test", "RMAE_test", "R2_test"]
]
reg_metrics.sort_values(by="RMSE_test").style.background_gradient(
    cmap="viridis", low=1, high=0.3, subset=["RMSE_train", "RMSE_test"]
).background_gradient(cmap="plasma", low=0.3, high=1, subset=["RMAE_test", "R2_test"])

,RMSE_train,RMSE_test,RMAE_test,R2_test
linear,17063.971412,17203.667235,118.717098,0.005924
linear_interact,17063.952659,17208.238142,118.733715,0.005395
ridge,17066.703394,17221.105113,118.713058,0.003907
linear_poly,17040.044207,17261.133005,119.105161,-0.000728
random_forest,14849.526391,17753.148423,121.331834,-0.058592
knn,15454.098490,18413.404485,123.492272,-0.138796
decision_tree,15686.930949,19256.171824,125.278504,-0.245425
mlp,61821.367865,61658.668659,243.300452,-11.769266


In [214]:
best_model = str(reg_metrics.sort_values(by="RMSE_test").iloc[0].name)

display(best_model)

'linear'

In [215]:
pd.concat(
    [
        X_train,
        y_train,
        pd.Series(
            class_models[best_model]["train_preds"],
            index=y_train.index,
            name="DensityPred",
        ),
    ],
    axis=1,
).head(5)

,Store_Area,Items_Available,Daily_Customer_Count,Store_Sales,Store_Sales,DensityPred
Store ID,,,,,,
120,1682,2017,780,49390,49390,60812.020548
266,1240,1492,940,66840,66840,57756.583567
356,1269,1511,500,54360,54360,57552.803165
877,1259,1493,100,88270,88270,57323.444026
782,1549,1871,740,74080,74080,60206.041163


In [216]:
pd.concat(
    [
        X_test,
        y_test,
        pd.Series(
            class_models[best_model]["preds"],
            index=y_test.index,
            name="DensityPred",
        ),
    ],
    axis=1,
).head(5)

,Store_Area,Items_Available,Daily_Customer_Count,Store_Sales,Store_Sales,DensityPred
Store ID,,,,,,
85,1287,1525,1200,90180,90180,57484.078439
689,1160,1401,900,35360,35360,57308.844819
143,1214,1456,530,48950,48950,57442.911147
795,1057,1280,610,41050,41050,56631.944802
745,854,1018,660,77740,77740,54682.174907


In [218]:
from sklearn.model_selection import GridSearchCV

optimized_model_type = "linear"
random_forest_model = class_models[optimized_model_type]["pipeline"]

parameters = {"alpha": [1, 10, 100, 290, 500],
              "fit_intercept": [True, False],
              "solver": ['svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'], 
}

gs_optomizer = GridSearchCV(
    estimator=random_forest_model, param_grid=parameters, n_jobs=-1
)

gs_optomizer.fit(X_train, y_train.values.ravel())
gs_optomizer.best_params_

ValueError: Invalid parameter 'alpha' for estimator Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('features_preprocessing',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('prepocessing_num',
                                                                   Pipeline(steps=[('imputer',
                                                                                    SimpleImputer(strategy='median')),
                                                                                   ('scaler',
                                                                                    StandardScaler())]),
                                                                   ['Store_Area',
                                                                    'Items_Available'])],
                                                    verbose_feature_names_out=False)),
                       